In [6]:
import pandas as pd
import os
import pickle

In [2]:
charts = os.listdir('data/CHARTS')
length_of_time_windows = [5, 15, 30, 60, 240, 1440]
preferred_time = 1440

In [3]:
amazon_stock_chart = pd.DataFrame()
apple_stock_chart = pd.DataFrame()
if 'AMAZON' + str(preferred_time) + ".csv" in charts:
    headers = ["date", "time", "open", "high", "low", "close", "volume"]
    amazon_stock_chart = pd.read_csv('data/CHARTS/' + 'AMAZON' + str(preferred_time) + ".csv", names=headers)
    amazon_stock_chart['upward_trend'] = (amazon_stock_chart['close'] - amazon_stock_chart['open'] > 0)
if 'APPLE' + str(preferred_time) + ".csv" in charts:
    headers = ["date", "time", "open", "high", "low", "close", "volume"]
    apple_stock_chart = pd.read_csv('data/CHARTS/' + 'APPLE' + str(preferred_time) + ".csv", names=headers)
    apple_stock_chart['upward_trend'] = (apple_stock_chart['close'] - apple_stock_chart['open'] > 0)
apple_stock_chart

,date,time,open,high,low,close,volume,upward_trend
0,2014.06.30,00:00,93.159,94.194,93.028,93.460,2799,True
1,2014.07.01,00:00,93.973,94.546,93.611,93.993,4787,True
2,2014.07.02,00:00,94.344,94.515,93.540,93.902,4476,False
3,2014.07.03,00:00,94.113,94.545,93.661,94.495,3454,True
4,2014.07.07,00:00,94.936,96.423,94.936,96.403,4638,True
...,...,...,...,...,...,...,...,...
1138,2019.01.28,00:00,156.250,156.490,153.650,155.920,62867,False
1139,2019.01.29,00:00,156.470,161.620,150.980,160.540,66595,True
1140,2019.01.30,00:00,161.680,166.110,160.220,165.660,70344,True
1141,2019.01.31,00:00,166.100,168.980,164.530,166.770,62821,True


In [4]:
amazon_stock_chart

,date,time,open,high,low,close,volume,upward_trend
0,2014.09.29,00:00,320.12,323.60,319.50,321.82,7187,True
1,2014.09.30,00:00,320.81,323.38,318.60,322.44,8364,True
2,2014.10.01,00:00,321.27,321.47,315.52,317.46,12002,False
3,2014.10.02,00:00,316.16,318.40,311.31,318.40,15487,True
4,2014.10.03,00:00,321.29,325.10,319.50,322.61,11113,True
...,...,...,...,...,...,...,...,...
1089,2019.01.28,00:00,1656.96,1662.25,1614.07,1633.73,41878,False
1090,2019.01.29,00:00,1638.23,1639.42,1584.97,1612.87,39136,False
1091,2019.01.30,00:00,1616.53,1685.61,1616.53,1684.04,39652,True
1092,2019.01.31,00:00,1703.57,1776.99,1670.01,1709.98,51953,True


In [5]:
with open('pkls/amazon_stock_chart.pkl', 'wb') as f:
    pickle.dump(apple_stock_chart, f)
    print("done")
with open('pkls/apple_stock_chart.pkl', 'wb') as f:
    pickle.dump(apple_stock_chart, f)
    print("done")

done
done
